In [2]:
def loatDataSet():
    return [[1,3,4],[2,3,5],[1,2,3,5],[2,5]]
def createC1(dataSet):
    C1 = []
    for transaction in dataSet:
        for item in transaction:
            if [item] not in C1:
                C1.append([item])
    C1.sort()
    return list(map(frozenset,C1))

In [5]:
def scanD(D,Ck,minSuppoort):
    ssCnt ={}
    for tid in D:
        for c in Ck:
            if c.issubset(tid):
                if c not in ssCnt:
                    ssCnt[c]=1
                else:
                    ssCnt[c]+=1
    numItems = float(len(D))
    retList = []
    supportData = {}
    for key in ssCnt:
        support = ssCnt[key]/numItems
        if support >= minSuppoort:
            retList.insert(0,key)
        supportData[key]=support
    return retList,supportData

In [8]:
def aprioriGen(Lk,k):
    retList = []
    lenLk = len(Lk)
    for i in range(lenLk):
        for j in range(i+1,lenLk):
            L1 = list(Lk[i])[:k-2]
            L2 = list(Lk[j])[:k-2]
        if L1==L2:
            retList.append(Lk[i]|Lk[j])
    return retList

def apriori(dataSet,minSupport=0.5):
    C1 = createC1(dataSet)
    D = list(map(set,dataSet))
    L1,supportData = scanD(D,C1,minSupport)
    L = [L1]
    k = 2
    while len(L(k-2))>0:
        Ck = aprioriGen(L[k-2],k)
        Lk,supK = scanD(D,Ck,minSupport)
        supportData.update(supK)
        L.append(Lk)
        k += 1
    return L,supportData
def calConf(freqSet,H,supportData,brl,minConf=0.7):
    pruneH = []
    for conseq in H:
        conf = supportData[freqSet]/supportData[freqSet-conseq]
        if conf >= minConf:
            print(freqSet-conseq,'-->',conseq,'conf:',conf)
            brl.append((freqSet-conseq,conseq,conf))
            pruneH.append(conseq)
    return pruneH


def rulesFromConseq(freqSet,H,supportData,brl,minConf=0.7):
    m = len(H[0])
    if len(freqSet)>(m+1):
        Hmpl = aprioriGen(H,m+1)
        Hmpl = calConf(freqSet,Hmpl,supportData,brl,minConf)
        if len(Hmpl)>1:
            rulesFromConseq(freqSet,Hmpl,supportData,brl,minConf)

def generateRules(L,supportData,minConf=0.7):
    bigRuleList=[]
    for i in range(1,len(L)):
        for freqSet in L[i]:
            H1 = [frozenset([item]) for item in freqSet]
            if i>1:
                rulesFromConseq(freqSet,H1,supportData,bigRuleList,minConf)
            else:
                calConf(freqSet,H1,supportData,bigRuleList,minConf)
    return bigRuleList

In [11]:
class treeNode:
    def __init__(self,nameValue,numOccur,parentNode):
        self.name = nameValue
        self.count = numOccur
        self.nodeLink = None
        self.parent = parentNode
        self.children={}
    def inc(self,numOccur):
        self.count+=numOccur
    def displayFPTree(self,ind=1):
        print(" "*ind,self.name," ",self.count)
        for child in self.children.values():
            child.displayFPTree(ind+1)
def createTree(dataSet,minsup=1):
    headerTable = {}
    for trans in dataSet:
        for item in trans:
            headerTable[item]=headerTable.get(item,0)+dataSet[trans]
    for item in list(headerTable.keys()):
        if headerTable[item]<minsup:
            del (headerTable[item])
    freqItemSet = set(headerTable.keys())
    if len(freqItemSet)==0:
        return None,None
    for item in headerTable:
        headerTable[item] = [headerTable[item],None]
    retTree = treeNode('Null Set',1,None)
    for trans,count in dataSet.items():
        localD = {}
        for item in trans:
            if item in trans:
                if item in freqItemSet:
                    localD[item] = headerTable[item][0]
            if len(localD)>0:
                orderedItems = [v[0] for v in sorted(localD.items(),key=lambda p:(p[1],-ord(p[0])),reverse=True)]
                updateTree(orderedItems,retTree,headerTable,count)
    return retTree,headerTable
def updateTree(items,inTree,headerTable,count):
    if items[0] in inTree.children:
        inTree.children[items[0]].inc(count)
    else:
        inTree.children[items[0]]=treeNode(items[0],count,inTree)
        if headerTable[items[0]][1]==None:
            headerTable[items[0]][1]=inTree.children[items[0]]
        else:
            updateHeadera(headerTable[items[0]][1],inTree.children[items[0]])
    if len(items)>1:
        updateTree(items[1:],inTree.children[items[0]],headerTable,count)
def updateHeadera(nodeToTest,targetNode):
    while (nodeToTest,nodeLink!=None):
        nodeToTest = nodeToTest.nodeLink
    nodeToTest.nodeLink = targetNode

In [12]:
def ascendTree(leafNode,prefixPath):
    if leafNode.parent != None:
        prefixPath.append(leafNode.name)
        ascendTree(leafNode.parent,prefixPath)
def findPrefixpath(treeNode):
    condPats = {}
    while treeNode!=None:
        prefixPath = []
        ascendTree(treeNode,prefixPath)
        if len(prefixPath)>1:
            condPats[frozenset(prefixPath[1:])]=treeNode.count
        treeNode = treeNode.nodeLink
    return condPats

In [13]:
def mineTree(inTree,headerTable,minsup,preFix,freqItemList):
    bigL = [v[0] for v in sorted(headerTable.items(),key=lambda p :p[1][0])]
    for basePat in bigL:
        newFreqSet = preFix.copy()
        newFreqSet.add(basePat)
        freqItemList.append(newFreqSet)
        condPattBase = findPrefixpath(headerTable[basePat][1])
        myCondTree,myHead = createTree(condPattBase,minsup)
        if myHead !=None:
            print('conditional tree for :',newFreqSet)
            myCondTree.displayFPTree()
            mineTree(myCondTree,myHead,minsup,newFreqSet,freqItemList)

In [15]:
def loadData():
    simpDat = [['r','z','h','j','p'],
               ['z','y','x','w','v','u','t','s'],
               ['z'],
               ['r','x','n','o','s'],
               ['y','r','x','z','q','t','p'],
               ['y','z','x','e','q','s','t','m']]
    return simpDat
def createInitSet(dataSet):
    retDict = {}
    for trans in dataSet:
        retDict[frozenset(trans)] = 1
    return retDict


In [16]:
dataSet = loadData()
initSet = createInitSet(dataSet)
minsup=3